In [ ]:
pip install pycvesearch cve_searchsploit cpe bs4 

In [ ]:
from bs4 import BeautifulSoup
import os
import pandas as pd
from io import StringIO
#import ipaddress
from auxiliary import aux


In [ ]:
alvos = ['35.199.110.191','54.196.179.100']

In [ ]:
resultados = os.path.join(os.getcwd(),'03-resultados_STEPES-TR')

## Passo 1 - Coletar Dados

#### A1.2 - Total

In [ ]:
# busca CVE diretamente na API ``https://cve.circl.lu/api/cve/''
from cve import cve
# busca CVE a partir de contêiner, endereço ``https://github.com/cve-search/CVE-Search-Docker.git'
from pycvesearch import CVESearch
cve_search = CVESearch('http://192.168.0.254:5000') 

In [ ]:
# busca Exploit
import cve_searchsploit as CS

## Passo 2 - Avaliar Vulnerabilidades

#### A2.1 Parcial

In [ ]:
# Obter serviços web
nmap_cmd = ['nmap -p0-65535 -T5 -Pn -sV -sT --min-rate 1000 ',
            'nmap -sS -v -Pn -A --open --script=vuln '
            #'nmap -sU -A -PN -n -pU:19,53,123,161 --script=ntp-monlist,dns-recursion,snmp-sysdescr ',
            #'nmap --script=mysql-brute '
            ]
if os.path.exists(resultados) is False:
    os.mkdir(resultados)
os.chdir(resultados)
for alvo in alvos:
    #print('############# ',alvo,' #############')
    for cmd in range(0,len(nmap_cmd)):
        tmp_cmd = nmap_cmd[cmd] + '-oX ' + os.path.join(resultados, '01-nmap_result+') +  alvo + '+_' + str(cmd+1) + '.xml ' + alvo
        print(tmp_cmd)
        #os.system(tmp_cmd)
os.chdir('..')

#### A2.2 Parcial

In [ ]:
os.chdir(resultados)
for alvo in alvos:
    #print('########\nIniciando alvo: '+alvo+'\n########')
    try:
        port_list = aux.get_port('01-nmap_result+' + alvo + '+_1.xml')
    except:
        None
    try:
        os_cpe,port_srv,name_srv,cpe_srv = aux.get_port_info('01-nmap_result+' + alvo + '+_2.xml')
    except:
        None
    port_list.extend(port_srv)
    port_list = sorted(set(port_list))
    data_cpe = {'alvo': alvo, 
                'os': os_cpe, 
                'portas': port_list,
                'srv': {'name': name_srv,
                        'cpe': cpe_srv,
                        'port': port_srv}}
    (pd.json_normalize(data_cpe)).to_json('02-cpe+' + alvo + '.json')
os.chdir('..')

In [ ]:
pd_a22 = pd.DataFrame(None)
for alvo in alvos:
    pd_a22 = pd.concat(
        [pd_a22, pd.read_json(os.path.join(resultados,'02-cpe+' + alvo + '.json'))
        ], ignore_index=True)
pd_a22

#### A2.1 Parcial (Continuação)

In [ ]:
#Executar verificadores locais para detectar vulnerabilidades (SHAKIBAZAD, 2019).
# Nikto2
os.system('apt-get update && apt-get install -y perl libnet-ssleay-perl openssl libauthen-pam-perl libio-pty-perl')
resultados_nikto = os.path.join(resultados,'2.1-nikto')
if os.path.exists(resultados_nikto) is False:
    os.mkdir(resultados_nikto)
os.chdir('/home/suporte/work/nikto/program/')
for x in pd_a22.index:
    print('Alvo: ', pd_a22.loc[x].iloc[0], ' Portas: ', len(pd_a22.loc[x].iloc[2]))
    for y in range(0,len(pd_a22.loc[x].iloc[2])):
        tmp_cmd1 = pd_a22.loc[x].iloc[0] + ':' + (pd_a22.loc[x].iloc[2])[y]
        tmp_cmd2 = pd_a22.loc[x].iloc[0] + '_' + (pd_a22.loc[x].iloc[2])[y]
        tmp_cmd3 = ' -o '+ os.path.join(resultados_nikto,'2.1-nikto+' + tmp_cmd2 + '.txt')
        tmp_cmd = 'perl nikto.pl -host ' + str(tmp_cmd1) + str(tmp_cmd3)
        #os.system(tmp_cmd)
        print(tmp_cmd)
os.chdir(resultados)
os.chdir('..')

In [ ]:
# Movendo relatórios sem dados
resultados_nikto_nweb = os.path.join(resultados,'2.1-nikto-nweb')
if os.path.exists(resultados_nikto_nweb) is False:
    os.mkdir(resultados_nikto_nweb)
os.chdir(resultados_nikto)
arqs = os.listdir()
for arq in arqs:
    if(arq[:3] == '2.1'):
        f = open(arq, "r").readlines()
        try:
            if(str(f[2][0:3]) == '+ T'):
                None
        except:
            os.rename(arq,os.path.join(resultados_nikto_nweb,arq))
    else:
        print(arq)
os.chdir(resultados)
os.chdir('..')

#### A2.3 Parcial

In [ ]:
os.chdir(resultados) #add cpe cpe:/:microsoft:windows_7:sp1
for alvo in alvos:
    pd_cve = pd.DataFrame(None)
    print('########\nIniciando alvo: '+alvo+'\n########')
    pd_a23 = pd.read_json('02-cpe+' + alvo + '.json')
    list_cpe = aux.clean_list_cpe(pd_a23['srv.cpe'].loc[0],'a')
    list_cpe_os = aux.clean_list_cpe(pd_a23['srv.cpe'].loc[0],'o')
    list_cpe_os.extend(aux.clean_list_cpe(pd_a23['os'].loc[0],'o'))
    list_cpe.extend(list_cpe_os)
    for cpe_id in list_cpe:
        if(cpe_id != 'unknown'):
            print('Iniciando processo no CPE: '+ cpe_id)
            w = cve_search.search(aux.get_cpe(cpe_id))
            if(w['total'] > 0):
                pd_a23_cve = pd.json_normalize(w['results'])
                pd_a23_cve = pd_a23_cve[
                    ['id','cvss','cwe','summary','references',
                     'access.authentication','access.complexity','access.vector',
                     'impact.availability','impact.confidentiality','impact.integrity']]
                data_cve = {'alvo': alvo, 
                        'cpe': cpe_id,
                        'cve': pd_a23_cve.to_json()}
                pd_cve = pd.concat([pd_cve,pd.json_normalize(data_cve)], ignore_index=True)
            else:
                print(aux.get_cpe(cpe_id), 'sem CVE!')
            
    if(not (pd_cve.empty)):
        pd_cve.to_json('03-cve+' + alvo + '.json')
os.chdir('..')

In [ ]:
pd_a23 = pd.DataFrame(None)
for alvo in alvos:
    try:
        pd_a23= pd.concat(
        [pd_a23,pd.read_json(os.path.join(resultados,'03-cve+' + alvo + '.json'))], ignore_index=True)
    except:
        None
pd_a23

#### A2.4 Parcial

In [ ]:
from misp import misp
# CVE com exploit no MISP
relative_path = 'attributes/restSearch'
body = {
    "returnFormat": "json",
    "type": "vulnerability"}

dados = misp.pymisp.direct_call(relative_path, body)
misp_dados = pd.DataFrame(dados['Attribute'])
print('Quantidade de CVE com Exploit disponível: ',len(misp_dados))
misp_dados[['event_id','category','type','value']].head()

In [ ]:
# Criar tag
misp.pymisp.add_tag({"name": "Exploit Available", 
                     "colour": "#22681c", 
                     "exportable": 'true', 
                     "org_id": "0", 
                     "user_id": "0", 
                     "hide_tag": 'false'})
########################################################################


# Aplicar tag em atributos ou eventos no MISP
name_tag = "Exploit Available"
for x in misp_dados.index:
    uuid_event = misp_dados.iloc[x].loc['uuid']
    misp.pymisp.tag(misp_entity=uuid_event,tag=name_tag)

########################################################################
# Criar evento com o alvo



## Passo 3 - Avaliar Riscos

#### A3.1 Parcial

#### A3.2 Parcial

In [ ]:
pd_a32 = pd.DataFrame(None)
for alvo in alvos:
    try:
        pd_a32= pd.concat(
        [pd_a32,pd.read_json(os.path.join(resultados,'03-cve+' + alvo + '.json'))], ignore_index=True)
    except:
        None
pd_a32

In [ ]:
x=0
y=1
while x < (len(pd_a32)-1):
    if(str(pd_a32.loc[x].iloc[0]) != str(pd_a32.loc[x+1].iloc[0])):
        print('Alvo = ',pd_a32.loc[x].iloc[0],'CPE = ',y)
        y=1
    x = x+1
    y = y+1
print('Alvo = ',pd_a32.loc[x-1].iloc[0],'CPE = ',y)

In [ ]:
tmp_cve = pd.read_json(pd_a32.loc[0].iloc[2])
tmp_cve.sort_values(["cvss"], ascending=True, inplace=True)
print(len(tmp_cve))

#tmp_cve.drop_duplicates(ignore_index=True,inplace=True)
print((tmp_cve))

x = 0
tmp_alto = 0
tmp_medio = 0
while x < (len(pd_a32)-1):
    if(str(pd_a32.loc[x].iloc[0]) != str(pd_a32.loc[x+1].iloc[0])):
        print('Alvo = ',pd_a32.loc[x].iloc[0],'Alto = ',tmp_alto,'Médio = ',tmp_medio)
        tmp_alto = 0
        tmp_medio = 0
    tmp_cve = pd.read_json(pd_a32.loc[x].iloc[2])
    tmp_cve_alto,tmp_cve_medio,tmp_cve_baixo = aux.get_len(tmp_cve) #aux.
    tmp_alto = tmp_alto + tmp_cve_alto
    tmp_medio = tmp_medio + tmp_cve_medio
    x = x+1
print('Alvo = ',pd_a32.loc[x-1].iloc[0],'Alto = ',tmp_alto,'Médio = ',tmp_medio)

In [ ]:
tmp_cve = pd.read_json(pd_a32.loc[x].iloc[2])
tmp_cve

In [ ]:
y = 0
lista_acesso = []
while y < (len(pd_a32)-1):
    if(str(pd_a32.loc[y].iloc[0]) != str(pd_a32.loc[y+1].iloc[0])):
        print('## Alvo = ',pd_a32.loc[y].iloc[0],'\n\tAcesso = ',lista_acesso)
        lista_acesso = []
    tmp_cve = pd.read_json(pd_a32.loc[y].iloc[2])
    for x in tmp_cve.index:
        tmp_acesso = 'Complexidade='+tmp_cve['access.complexity'].loc[x]+' Vetor='+tmp_cve['access.vector'].loc[x]
        if(tmp_acesso not in lista_acesso):
            lista_acesso.append(tmp_acesso)
    y=y+1
print('## Alvo = ',pd_a32.loc[y].iloc[0],'\n\tAcesso = ',lista_acesso)

In [ ]:
tmp_cve['access.complexity'].loc[33]

#### A3.3 Parcial

In [ ]:
# print com alvo, cwe(sem duplicatas), descrição

## Passo 4 - Mitigar Vulnerabilidades e Riscos

#### A4.2 Parcial

In [ ]:
pd_a42 = pd.DataFrame(None)
for alvo in alvos:
    try:
        pd_a42= pd.concat(
        [pd_a42,pd.read_json(os.path.join(resultados,'03-cve+' + alvo + '.json'))], ignore_index=True)
    except:
        None
pd_a42.head()

In [ ]:
from misp import misp
# Alvo com exploit postado no MISP
relative_path = 'attributes/restSearch'
y = 0
while y < (len(pd_a42)-1):
    tmp_cve = pd.read_json(pd_a42.loc[y].iloc[2])
    for x in range(0,len(tmp_cve)):
        dados = misp.pymisp.direct_call(relative_path, 
                                    {"returnFormat": "json","value": tmp_cve['id'].loc[x]})
        misp_dados = pd.DataFrame(dados['Attribute'])
        if(not (misp_dados.empty)):
            print('Alvo: ',pd_a42.loc[y].iloc[0],'CVE: ',tmp_cve['id'].loc[x],', com EXPLOIT!')
    y=y+1

In [ ]:
from misp import misp
# Alvo com exploit postado no MISP
relative_path = 'events/nids/snort/download/'
dados = misp.pymisp.direct_call(relative_path, 
                            {"returnFormat": "json","to_ids": True})
data_pd = pd.json_normalize(dados)

In [ ]:
print('Quantidade de linhas de dados = ', len(data_pd))
df = pd.json_normalize(data_pd['Event.Attribute'].loc[0]) #Apenas 1 linha de evento
df[['category','to_ids','value']]

In [ ]:
dados[0]

In [ ]:
misp.search('attibutes','yara')

## Passo 5 - Apresentar Relatório de Ameaças

#### A5.1 Parcial

#### A5.2 Parcial

In [ ]:
pd_a22 = pd.DataFrame(None)
for alvo in alvos:
    pd_a22 = pd.concat(
        [pd_a22, pd.read_json(os.path.join(resultados,'02-cpe+' + alvo + '.json'))
        ], ignore_index=True)
pd_a22.head()

In [ ]:
pd_a23 = pd.DataFrame(None)
for alvo in alvos:
    try:
        pd_a23= pd.concat(
        [pd_a23,pd.read_json(os.path.join(resultados,'03-cve+' + alvo + '.json'))], ignore_index=True)
    except:
        None
pd_a23.head()